In [1]:
import sys
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier



from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB

import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import GridSearchCV



In [2]:
t = pd.read_csv("./reports/data.csv")
t = t.dropna()

In [3]:
__type = "search"#sys.argv[1]
t[__type] =  t["labels"].str.contains(__type)
t[__type] = pd.to_numeric(t[__type], downcast="signed")

In [4]:

l =t['ps']= (t['problem-statement'].astype(str) +t['output-spec']) .tolist()
res1=[]
regex = re.compile("^a-zA-Z0-9\ '" )
def clean_s(input_str):
    return regex.sub(' ', input_str)

for i in range(len(l)):
    try:
        res1.append(clean_s(l[i]))
    except:
        res1.append("")

In [5]:
output = t[[__type,'ps']]
output = output.dropna()

output[__type] = output[__type].astype(int)

a = output.loc[output[__type] == 1.0]
b = output.loc[output[__type] == 0.0]
b = b.sample(n=len(a), random_state = 42)
output = a.append(b)

In [6]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.optimizers import Adam

max_features = 20000
batch_size = 32
embedding_dims = 500
filters = 250
kernel_size = 3
hidden_dims = 2500
epochs = 2
maxlen = 5000

temp =  [hashing_trick(a, max_features) for a in output['ps'].tolist()]

temp= sequence.pad_sequences(temp, maxlen=maxlen);
#pad2= sequence.pad_sequences(temp2);
#temp = np.concatenate((sequence.pad_sequences(temp1), sequence.pad_sequences(temp2)), axis = 1) #np.concatenate((a, b.T), axis=1)
x_train, x_test, y_train, y_test = train_test_split(temp,output[__type], test_size=0.2,random_state=42)


/home/nikepupu/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
print(temp.shape)

(814, 5000)


In [8]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions


model.add(Embedding(maxlen, 128))
model.add(Dropout(0.5))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())



# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.5))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
validation_data=(x_test, y_test))

y_pred = model.predict(x_test)
y_pred = [round(value) for value in y_pred]
print(classification_report(y_test, y_pred))

Build model...
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 651 samples, validate on 163 samples
Epoch 1/20
651/651 [==============================] - 4s 7ms/step - loss: 0.6976 - acc: 0.5146 - val_loss: 0.7109 - val_acc: 0.4601
Epoch 2/20
651/651 [==============================] - 3s 4ms/step - loss: 0.6943 - acc: 0.4992 - val_loss: 0.6932 - val_acc: 0.4479
Epoch 3/20
651/651 [==============================] - 3s 4ms/step - loss: 0.6967 - acc: 0.4916 - val_loss: 0.6923 - val_acc: 0.5276
Epoch 4/20
651/651 [==============================] - 3s 4ms/step - loss: 0.6851 - acc: 0.5530 - val_loss: 0.6950 - val_acc: 0.4479
Epoch 5/20
651/651 [==============================] - 3s 4ms/step - loss: 0.6625 - acc: 0.5837 - val_loss: 0.6820 - val_acc: 0.5828
Epoch 6/20
651/651 [==============================] - 3s 4ms/step - loss: 0.6240 - acc: 0.6406 - val_loss: 0.7146 - val_acc: 0.5706
Epoch 7/20
651/651 [==============================] - 3s 4ms/ste

In [9]:
#greedy
model.save('keras_search_classifier.h5')